In [1]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup

# Парсинг 
Вдохновившись новостями по нобелевской неделе, для парсинга я выбрал сайт Naked Science, который содержит научнопопулярные статьи и новости из мира науки. Из каждой статьи можно вытащить ее название, автора, аннотацию, индекс важности (оценка автором важности исследования/открытия для науки), дату публикации, сам текст статьи, хэштеги и количество просмотров в качестве целевой переменной.

In [2]:
user_ag = UserAgent()
main_link = 'https://naked-science.ru/article/page/2/'

In [3]:
response = requests.get(main_link, headers={'User-Agent': user_ag.random})
response.ok

True

In [5]:
soup = BeautifulSoup(response.content, 'html.parser')

In [12]:
content = soup.find('div', attrs={'class': 'content'}).find('div', attrs={'class': 'news-items'})
print(content.prettify())

<div class="news-items">
 <div class="news-item grid">
  <div class="news-item-left with-bookmark">
   <div class="meta-items grid">
    <div class="meta-items-l grid">
     <div class="meta-item">
      <span class="echo_date">
       3 октября, 13:32
      </span>
     </div>
     <!-- edit start -->
     <div class="meta-item meta-item_author" onclick='document.location.href = "https://naked-science.ru/user/75829"; return false'>
      Юлия Трепалина
     </div>
     <!-- edit end -->
     <div class="meta-items-r meta-hidden">
      <div class="meta-item">
       <div class="fvc-view" data-view-count-id="1118548" data-view-count-update="0">
        <span class="fvc-count">
         2 104
        </span>
        <span class="fvc-icon">
         <svg height="10px" version="1.1" viewbox="0 0 16 10" width="16px" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
          <g fill="none" fill-rule="evenodd" stroke="none" stroke-width="1">
           <g fill="

In [ ]:
'news-item-left', 'with-bookmark'

In [16]:
news = content.find_all(lambda tag: tag.name == 'div' and tag.get('class') == ['news-item-left', 'with-bookmark'])
print(len(news))
print(news[-1].prettify())

20
<div class="news-item-left with-bookmark">
 <div class="meta-items grid">
  <div class="meta-items-l grid">
   <div class="meta-item">
    <span class="echo_date">
     2 октября, 10:24
    </span>
   </div>
   <!-- edit start -->
   <div class="meta-item meta-item_author" onclick='document.location.href = "https://naked-science.ru/user/19"; return false'>
    ТюмГУ
   </div>
   <!-- edit end -->
   <div class="meta-items-r meta-hidden">
    <div class="meta-item">
     <div class="fvc-view" data-view-count-id="1118213" data-view-count-update="0">
      <span class="fvc-count">
       718
      </span>
      <span class="fvc-icon">
       <svg height="10px" version="1.1" viewbox="0 0 16 10" width="16px" xmlns="http://www.w3.org/2000/svg" xmlns:xlink="http://www.w3.org/1999/xlink">
        <g fill="none" fill-rule="evenodd" stroke="none" stroke-width="1">
         <g fill="#999999" fill-rule="nonzero" transform="translate(-43.000000, -5.000000)">
          <path d="M51.0031165,15 C46

In [23]:
obj = news[2]
date = obj.find('span', attrs={'class': 'echo_date'}).string
date.string

'3 октября, 12:20'

In [25]:
def parse_date(date):
    ...

In [31]:
views = int(''.join(obj.find('span', attrs={'class': 'fvc-count'}).string.split()))
views

6766